In [0]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lm=WordNetLemmatizer()
from nltk import pos_tag

In [0]:
import pandas as pd

p=pd.read_csv("/0000000000002747_training_twitter_x_y_train.csv")
c=p.iloc[:,1:2]
ytrain=p['airline_sentiment'].to_list()
doc=p['text'].to_list()
c=pd.read_csv("/0000000000002747_test_twitter_x_test.csv")
xtest=c['text'].to_list()


In [0]:
def getsimplepos(tag):
    if tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    nltk.download('stopwords')
from nltk.corpus import stopwords
stops=set(stopwords.words("english"))
import string
p=list(string.punctuation)
stops.update(p)
def clean(words):
    op=[]
    for w in words:
        if w not in stops:
            pos=pos_tag([w])
            cw=lm.lemmatize(w,pos=getsimplepos(pos[0][1]))
            op.append(cw.lower())
    return op        
dcs=[clean(doc[s].split())for s in range(len(doc))]            
tests=[clean(xtest[s].split()) for s in range(len(xtest))]

In [0]:
for i in range(len(dcs)):
  d=dcs[i].pop(0)
for i in range(len(tests)):
  tests[i].pop(0)


In [0]:
dxs=[" ".join(i) for i in dcs]
testx=[" ".join(i) for i in tests]

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
countvec=CountVectorizer(max_features=3000,ngram_range=(1,2))
xtrainf=countvec.fit_transform(dxs)
xtrainf.todense()
xtestf=countvec.transform(testx)


In [65]:
from sklearn.naive_bayes import MultinomialNB
rf=MultinomialNB()
rf.fit(xtrainf,ytrain)
yp=rf.predict(xtestf)


array(['negative', 'negative', 'negative', ..., 'neutral', 'positive',
       'neutral'], dtype='<U8')

In [0]:
import numpy as np
np.savetxt("ypr.csv",yp,fmt='%s')